In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Plan and Execute Agents

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

# Overview

<center>
<img src="imgs/zghost_overview_agents.png" width="1200"/>
</center>

In this notebook we are progressing the concept of using [Agents](https://python.langchain.com/en/latest/modules/agents.html) with Vertex LLMs. The prior notebook showed how to establish agents that operate on one chain of thought along with one tool. In this notebook, we add more tools for the LLM to use in the retrieval process. 

**Plan and execute agents** 
* [Plan-and-execute Agents](https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html) accomplish an objective by first planning what to do, then executing the sub tasks
* This idea is largely inspired by [BabyAGI](https://github.com/yoheinakajima/babyagi) and the [Plan-and-Solve](https://arxiv.org/abs/2305.04091) paper.
* The planning is almost always done by an LLM.
* The execution is usually done by a separate agent (**equipped with tools**)

**Equipping LLMs with tools**

Tool refers to a 'capability' of an agent. This is an abstraction on top of a function that makes it easy for LLMs (and agents) to interact with it. Ex: Google search.

In the paper [Decoupling Reasoning from Observations for Efficient Augmented Language Models](https://arxiv.org/abs/2305.18323), the authors propose the idea of Augmented Language Models (ALMs), which combine the reasoning capabilities of Large Language Models (LLMs) with tools that allow for knowledge retrieval and action execution:  

* Existing ALM systems trigger LLM thought processes while pulling observations from these tools in an interleaved fashion
* Specifically, an LLM reasons to call an external tool, gets halted to fetch the tool's response, and then decides the next action based on all preceding response tokens
* Such a paradigm, though straightforward and easy to implement, often leads to huge computation complexity from redundant prompts and repeated execution

**We'll augment our previously created VectorStore Agent with the following tools:**
* Our GDELT index comprised of global news related to our `ACTOR`
* The [Google Trends](https://trends.google.com/trends/explore?hl=en) public dataset. This data source helps us understand what people are searching for, in real time. We can use this data to measure search interest in a particular topic, in a particular place, and at a particular time
* [Google Search API Wrapper](https://developers.google.com/custom-search/v1/overview) - to retrieve and display search results from web searches
* A calculator to help the LLM with math
* An [SQL Database Agent](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html) for interacting with SQL databases (e.g., BigQuery). As an example, an agent's plan may require it to search for trends in the Google Trends BigQuery table


### **Below is an image showing the user flow interacting with the multi-tool agent using a plan-execute strategy**:
<center>
<img src="imgs/user-flow-plan-execute.png" width="1000"/>
</center>


### **Here's an example of what the user output will look like in a notebook**:
<center>
<img src="imgs/agent_plan_execute_chain_output.png" width="1500"/>
</center>
---

### **Objective**

In this notebook, we:

1) Instantiate the agents we have seen in the prior notebook
2) Take out the agent's tools
3) Provide instruction for additional tools (e.g. SQL tool - point it to the correct table)
4) Build an agent that utilizes the multiple tools previously described
5) Provide a planner for the agent to convert it to a **Plan and Execute Agent**

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Support
* Vertex AI Matching Engine

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started
**Colab only:** Uncomment the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Make sure you edit the values below
Each time you run the notebook for the first time with new variables, you just need to edit the actor prefix and version variables below. They are needed to grab all the other variables in the notebook configuration.

In [1]:
# CREATE_NEW_ASSETS        = True # True | False
ACTOR_PREFIX             = "ggl"
VERSION                  = 'v1'

# print(f"CREATE_NEW_ASSETS  : {CREATE_NEW_ASSETS}")
print(f"ACTOR_PREFIX       : {ACTOR_PREFIX}")
print(f"VERSION            : {VERSION}")

ACTOR_PREFIX       : ggl
VERSION            : v1


### Load configuration settings from setup notebook
> Set the constants used in this notebook and load the config settings from the `00-env-setup.ipynb` notebook.

In [2]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

BUCKET_NAME              = f'zghost-{ACTOR_PREFIX}-{VERSION}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)

print(f"BUCKET_NAME        : {BUCKET_NAME}")
print(f"BUCKET_URI         : {BUCKET_URI}")


PROJECT_ID               = "wortz-project-352116"
PROJECT_NUM              = "679926387543"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "me-network"

CREATE_NEW_ASSETS        = "True"
ACTOR_PREFIX             = "ggl"
VERSION                  = "v1"
ACTOR_NAME               = "google"
ACTOR_CATEGORY           = "technology"

BUCKET_NAME              = "zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET     = "zghost-ggl-v1-wortz-project-352116-emd-dir"

BUCKET_URI               = "gs://zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET_URI = "gs://zghost-ggl-v1-wortz-project-352116-emd-dir"

VPC_NETWORK_FULL         = "projects/679926387543/global/networks/me-network"

ME_INDEX_NAME            = "vectorstore_ggl_v1"
ME_INDEX_ENDPOINT_NAME   = "vectorstore_ggl_v1_endpoint"
ME_DIMENSIONS            = "768"

MY_BQ_DATASET            = "zghost_ggl_v1"
MY_BQ_TRENDS_DATASET     = "zg

### Import Packages

In [3]:
import sys
import os
sys.path.append("..")

from zeitghost.agents.LangchainAgent import LangchainAgent
from zeitghost.vertex.LLM import VertexLLM
from zeitghost.vertex.Embeddings import VertexEmbeddings

from zeitghost.vertex.MatchingEngineCRUD import MatchingEngineCRUD
from zeitghost.vertex.MatchingEngineVectorstore import MatchingEngineVectorStore

from zeitghost.agents.Helpers import MyCustomHandler

In [4]:
# Google Cloud
from google.cloud import aiplatform as vertex_ai
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.aiplatform_v1 import IndexServiceClient, IndexEndpointServiceClient
from google.cloud import secretmanager

# langchain
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document
from langchain import PromptTemplate

from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.agents.tools import Tool
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

from langchain.tools import Tool
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks.manager import CallbackManager
from langchain import LLMMathChain

from langchain.sql_database import SQLDatabase
from langchain.utilities import GoogleSearchAPIWrapper

import pandas as pd
import uuid
import numpy as np
import json
import time
import io

from pydantic import BaseModel, Field

from IPython.display import display, Image, Markdown
from PIL import Image, ImageDraw
import logging
logging.basicConfig(level = logging.INFO)

Instantiate Google cloud SDK clients

In [5]:
storage_client = storage.Client(project=PROJECT_ID)

vertex_ai.init(project=PROJECT_ID,location=LOCATION)

# bigquery client
bqclient = bigquery.Client(
    project=PROJECT_ID,
    # location=LOCATION
)

## Instantiate the Vertex AI resources, Agents, and Tools

* We are creating agents to quickly access the tools
* The tools can be defined from the ground-up using [langchain tools](https://python.langchain.com/en/latest/modules/agents/tools.html)

Tools
* Custom Google Search API

### Matching Engine Agent Tools
Get matching engine `Index` and `IndexEndpoint` IDs

In [6]:
mengine = MatchingEngineCRUD(
    project_id=PROJECT_ID 
    , project_num=PROJECT_NUM
    , region=LOCATION 
    , index_name=ME_INDEX_NAME
    , vpc_network_name=VPC_NETWORK_FULL
)

In [7]:
ME_INDEX_RESOURCE_NAME, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()
ME_INDEX_ID=ME_INDEX_RESOURCE_NAME.split("/")[5]

print(f"ME_INDEX_RESOURCE_NAME  = {ME_INDEX_RESOURCE_NAME}")
print(f"ME_INDEX_ENDPOINT_ID    = {ME_INDEX_ENDPOINT_ID}")
print(f"ME_INDEX_ID             = {ME_INDEX_ID}")

ME_INDEX_RESOURCE_NAME  = projects/679926387543/locations/us-central1/indexes/2525068035606183936
ME_INDEX_ENDPOINT_ID    = projects/679926387543/locations/us-central1/indexEndpoints/5353047126618144768
ME_INDEX_ID             = 2525068035606183936


### Setup the Matching Engine VectorStore Retreiver 

In [8]:
REQUESTS_PER_MINUTE = 200 # project quota==300
vertex_embedding = VertexEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)

me = MatchingEngineVectorStore.from_components(
    project_id=PROJECT_ID
    , region=LOCATION
    , gcs_bucket_name=BUCKET_NAME
    , embedding=vertex_embedding
    , index_id=ME_INDEX_ID
    , endpoint_id=ME_INDEX_ENDPOINT_ID
    , k = 10
)

### Create VectorStore Agent tool
This tool will allow our LLM to retreive information from the Matching Engine index we've created with GDELT data 

In [9]:
vertex_langchain_agent = LangchainAgent()

agent_executor = vertex_langchain_agent.get_vectorstore_agent(
    vectorstore=me
    , vectorstore_name=f"news on {ACTOR_NAME}"
    , vectorstore_description=f"a vectorstore containing news articles and current events for {ACTOR_NAME}."
)

me_tools = agent_executor.tools

In [10]:
for t in me_tools:
    print(t.name)
    print(t.description)
    print('\n')

news on google
Useful for when you need to answer questions about news on google. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this. Input should be a fully formed question.


news on google_with_sources
Useful for when you need to answer questions about news on google and the sources used to construct the answer. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this.  Input should be a fully formed question. Output is a json serialized dictionary with keys `answer` and `sources`. Only use this tool if the user explicitly asks for sources.




## BigQuery Agent 

We will use the Google Trends tables we copied to our `MY_BQ_DATASET` in the `00-env-setup.ipynb` notebook

We have noticed that keeping the tables for BigQuery Agent grouped by dataset tends decrease the hallucination and keeps the agent from getting confused about which tables should be used to answer a question - this is why we have separated the GDELT tables and the BQ Public Trends tables in separate datasets during setup.

**Note on instruction tuning for langchain SQL tools**

In our experience, the following types of instructions are useful for helping the agents understand the prompt instructions
Also note that there are limiting descriptions in the tools that prevent the SQL agent from performing DDL (create new tables, etc..)

In [11]:
TRENDS_DATASET  = MY_BQ_TRENDS_DATASET
TRENDS_TABLE_ID = 'top_rising_terms'
print(f"MY_BQ_TRENDS_DATASET: {MY_BQ_TRENDS_DATASET}")

vertex_langchain_agent = LangchainAgent()
bq_agent = vertex_langchain_agent.get_bigquery_agent(PROJECT_ID)

MY_BQ_TRENDS_DATASET: zghost_ggl_v1_trends


Add additional instructions for the BQ Agent to help with information retrieval - feel free to experiment and change these depending on your data, use case, etc.

In [12]:
bq_agent_tools = bq_agent.tools

bq_agent_tools[0].description = bq_agent_tools[0].description + \
 f"""
  only use the schema {TRENDS_DATASET}
  NOTE YOU CANNOT DO OPERATIONS AN AN AGGREGATED FIELD UNLESS IT IS IN A CTE WHICH IS ALLOWED
  also - use a like operator for the term field e.g. WHERE term LIKE '%keyword%' 
  make sure to lower case the term in the WHERE clause
  be sure to LIMIT 100 for all queries
  if you don't have a LIMIT 100, there will be problems
 """

In [13]:
for t in bq_agent_tools:
    print(t.name)
    print(t.description)
    print('\n')

sql_db_query
Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', using schema_sql_db to query the correct table fields.
  only use the schema zghost_ggl_v1_trends
  NOTE YOU CANNOT DO OPERATIONS AN AN AGGREGATED FIELD UNLESS IT IS IN A CTE WHICH IS ALLOWED
  also - use a like operator for the term field e.g. WHERE term LIKE '%keyword%' 
  make sure to lower case the term in the WHERE clause
  be sure to LIMIT 100 for all queries
  if you don't have a LIMIT 100, there will be problems
 


sql_db_schema
Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling list_tables_sql_db first! Example Input: 'table1, table2, table3'


sql_db_list_

## Google Search API
The [Custom Search JSON API](https://developers.google.com/custom-search/v1/overview) lets you develop websites and applications to retrieve and display search results from Programmable Search Engine programmatically. With this API, you can use RESTful requests to get either web search or image search results in JSON format.

Follow instructions from the documentation to:
- Set up a Custom Search Engine (which will generate your CSE_ID) on Google - you will need to enable the API for your GCP project if you haven't used the Search API before
- Get API key from Credentials on Google Cloud Console (GOOGLE_API_KEY)
- Configure these using either environment variables in Jupyter notebook or if running locally you can use the sample .env file `.env.sample` rename as `.env` and set your values for the API Keys.

*WARNING*: Do not push environment variables or .env files to repository as these are sensitive information and should not be stored in a repository 

Environment Variables in Jupyter Notebooks

```python
%env GOOGLE_CSE_ID value
%env GOOGLE_API_KEY value
```

Uncomment the below cell if you are using a Vertex AI User Managed Notebook

In [19]:
# %env GOOGLE_CSE_ID your-searchengine-id-here
# %env GOOGLE_API_KEY your-api-key-here

## Build an Agent that has access to Multiple Tools
Now that we have created multiple tools (GDELT Vector DB, BigQuery Public Trends, Google Search API), let's create an agent with access to all of these tools to answer questions. 

Note for the LLM we are including stopwords to prevent agent hallucination

Steps:
1) Instantiate a langchain LLM with Vertex
2) Instantiate the calculator tools (LLMs are bad at math) and search tools
3) Create a base schema for the tool output - this prevents tool output hallucination on important chain keys, such as `action_input`
4) Create a `tools` object
5) Apply the base schema to the tools
6) Extend the print out the tools

In [24]:
llm = VertexLLM(
    temperature=0
    , stop=['Observation:']
    # , stop=None
    , strip=True
    , strip_chars=['\\', '\\\\', "`"]
    , max_output_tokens=1000
    , top_p=0.7
    , top_k=40
)

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

search = GoogleSearchAPIWrapper() #IF YOU DON'T PERIODICALLY REFRESH THIS YOU WILL NOT GET RESULTS

class CalculatorInput(BaseModel):
    question: str = Field()
    
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        args_schema = CalculatorInput
    ),
    Tool(
        name = "Google Search",
        description="Search Google for recent results.",
        func=search.run,
    ),
]

#### Combine all tools 

In [25]:
# Apply the correct args schema to the tools

class BaseSchema(BaseModel):
    # action: str = Field()
    action_input: str = Field()

new_tools_with_args_schema = []

for tool in me_tools + bq_agent_tools:
    tool.args_schema = BaseSchema
    new_tools_with_args_schema.append(tool)
    
tools.extend(new_tools_with_args_schema)

#### Inspect all of the tools available for our agent

When you inspect the tools, you should see something similar to:
```
Calculator
useful for when you need to answer questions about math


Google Search
Search Google for recent results.


news on google
Useful for when you need to answer questions about news on google. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this. Input should be a fully formed question.


news on google_with_sources
Useful for when you need to answer questions about news on google and the sources used to construct the answer. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this.  Input should be a fully formed question. Output is a json serialized dictionary with keys `answer` and `sources`. Only use this tool if the user explicitly asks for sources.


query_sql_db

    Input to this tool is a detailed and correct SQL query, output is a result from the database.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    
  only use the schema zghost_ggl_v1_trends
  NOTE YOU CANNOT DO OPERATIONS AN AN AGGREGATED FIELD UNLESS IT IS IN A CTE WHICH IS ALLOWED
  also - use a like operator for the term field e.g. WHERE term LIKE '%keyword%' 
...
    Always use this tool before executing a query with query_sql_db!
```

In [26]:
for t in tools:
    print(t.name)
    print(t.description)
    print('\n')

Calculator
useful for when you need to answer questions about math


Google Search
Search Google for recent results.


news on google
Useful for when you need to answer questions about news on google. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this. Input should be a fully formed question.


news on google_with_sources
Useful for when you need to answer questions about news on google and the sources used to construct the answer. Whenever you need information about a vectorstore containing news articles and current events for google. you should ALWAYS use this.  Input should be a fully formed question. Output is a json serialized dictionary with keys `answer` and `sources`. Only use this tool if the user explicitly asks for sources.


sql_db_query
Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. 

### Provide a planner for the agent to convert it to a **Plan and Execute Agent**

What this does
1) Loads a planner - this creates a plan to run mulitple chains by the tool based on the prompt provided
2) Load an plan-execute agent: https://python.langchain.com/en/latest/modules/agents/plan_and_execute.html
3) Note the tools being loaded to the executor and agent instantiation

In [27]:
def get_plan_and_execute_agent(llm=llm, tools=tools):
    model = llm
    planner = load_chat_planner(model)
    executor = load_agent_executor(model, tools, verbose=True)
    agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
    return agent

agent = get_plan_and_execute_agent(llm=llm, tools=tools)
# agent

## Agent Q/A

For the prompt, we will use a [Prompt Template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)
* An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.
* Think of it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts

Now, let's test some output of this new "super-agent"
* You should be able to see the plan output as the first line then the chains will get executed

In [28]:
template = """How does {actors} commitment to technology impact their brand perception? 
           Use all of the tools available to process this question. 
           Also, cite your sources.
           Summarize your findings."""

prompt = PromptTemplate(
    input_variables=["actors"],
    template=template,
)
prompt.format(actors=ACTOR_NAME)

agent(prompt.format(actors=ACTOR_NAME))



> Entering new  chain...
steps=[Step(value='Google is a technology company that has been committed to innovation since its inception.'), Step(value="This commitment to technology has been a key factor in the company's success."), Step(value="Google's commitment to technology has also had a positive impact on its brand perception."), Step(value='Google is seen as a leader in technology, and this has helped to build trust and credibility with consumers.'), Step(value="Google's commitment to technology has also helped to create a positive image for the company."), Step(value='Google is seen as a forward-thinking company that is always looking for new ways to improve its products and services.'), Step(value="Given the above steps taken, please respond to the users original question:\n  \n  Google's commitment to technology has had a positive impact on its brand perception. The company is seen as a leader in technology, and this has helped to build trust and credibility with consumers. Go

{'input': 'How does google commitment to technology impact their brand perception? \n           Use all of the tools available to process this question. \n           Also, cite your sources.\n           Summarize your findings.',
 'output': 'Action:\n\n{\n  "action": "Final Answer",\n  "action_input": "Google is a technology company that has been committed to innovation since its inception. This commitment to technology has been a key factor in the company\'s success. Google\'s commitment to technology has also had a positive impact on its brand perception. Google is seen as a leader in technology, and this has helped to build trust and credibility with consumers. Google is seen as a forward-thinking company that is always looking for new ways to improve its products and services."n}nn"\n}\n'}

#### Because the agent remembers the prior chains, it is usually a good idea to re-instantiate the agents (which we do below before calling another prompt)

In [29]:
agent = get_plan_and_execute_agent(llm=llm, tools=tools)

template = """What are trends for {actors} 's supply chain? 
           Use all of the tools available to process this question. 
           Also, cite your sources."""

prompt = PromptTemplate(
    input_variables=["actors"],
    template=template,
)
prompt.format(actors=ACTOR_NAME)

agent(prompt.format(actors=ACTOR_NAME))



> Entering new  chain...
steps=[Step(value='Go to Google\'s website and search for "supply chain".'), Step(value='Click on the link to the "Google Supply Chain" page.'), Step(value="Read the information on the page to learn about Google's supply chain."), Step(value='Use the information on the page to answer the question.'), Step(value='Cite the following sources:\n    * Google Supply Chain: https://www.google.com/about/company/supply-chain/\n    * Google Supply Chain Blog: https://supplychain.googleblog.com/\n<END_OF_PLAN>')]

> Entering new  chain...
Question: What is the supply chain?
Action:

{
  "action": "Google Search",
  "action_input": "supply chain"
}



> Finished chain.
*****

Step: Go to Google's website and search for "supply chain".

Response: Question: What is the supply chain?
Action:

{
  "action": "Google Search",
  "action_input": "supply chain"
}



> Entering new  chain...
Question: What is the supply chain?
Action:

{
  "action": "Google Search",
  "action_inpu

{'input': "What are trends for google 's supply chain? \n           Use all of the tools available to process this question. \n           Also, cite your sources.",
 'output': 'Action:\n\n{\n  "action": "news on google_with_sources",\n  "action_input": "google supply chain"\n}\n\n'}

In [30]:
agent = get_plan_and_execute_agent(llm=llm, tools=tools)

template = """What are some examples of the top trending news topics for the {actor_category} category? 
           Use all of the tools available to process this question. """

prompt = PromptTemplate(
    input_variables=["actor_category"],
    template=template,
)
prompt.format(actor_category=ACTOR_CATEGORY)

agent(prompt.format(actor_category=ACTOR_CATEGORY))



> Entering new  chain...
steps=[Step(value='Go to the Google News website.'), Step(value='Click on the Technology tab.'), Step(value='Scroll down to see the top trending news topics.\nGiven the above steps taken, please respond to the users original question:\n<END_OF_PLAN>')]

> Entering new  chain...
Question: What is the latest news on Google?
Action:

{
  "action": "news on google",
  "action_input": "What is the latest news on Google?"
}



> Finished chain.
*****

Step: Go to the Google News website.

Response: Question: What is the latest news on Google?
Action:

{
  "action": "news on google",
  "action_input": "What is the latest news on Google?"
}



> Entering new  chain...
Action:

{
  "action": "Google Search",
  "action_input": "technology"
}



> Finished chain.
*****

Step: Click on the Technology tab.

Response: Action:

{
  "action": "Google Search",
  "action_input": "technology"
}



> Entering new  chain...
Action:

{
  "action": "news on google",
  "action_input

{'input': 'What are some examples of the top trending news topics for the technology category? \n           Use all of the tools available to process this question. ',
 'output': 'Action:\n\n{\n  "action": "news on google",\n  "action_input": "What is the latest news on Google?"\n}\n\n'}